In [5]:
import numpy as np
import datajoint as dj
import time
import pymeshfix
import os
import datetime
import calcification_Module as cm

#for supressing the output
import os, contextlib
import pathlib
import subprocess

#for error counting
from collections import Counter

#for reading in the new raw_skeleton files
import csv

from meshparty import trimesh_io

#for filtering
import math
from pykdtree.kdtree import KDTree

In [6]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_pinky')
pinky = dj.create_virtual_module('pinky', 'microns_pinky')


In [8]:
schema.jobs & "table_name='__neuron_skeleton_stitched'" & "status ='error'"

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__neuron_skeleton_stitched,0c10b6debdba7acb2d50db8adda4f0ca,error,=BLOB=,ValueError: string is not a file: /notebooks/Pass_2_Sk_Neurons/copies/pymesh_NEURONS/neuron_648518346349501216_piece_0_mls.off,=BLOB=,celiib@10.28.0.81,f031c4c9faf9,33642,732258,2019-07-14 03:51:09
__neuron_skeleton_stitched,11d28c04739ffed6715289f434fd8d5b,error,=BLOB=,Exception: neuron648518346349500742 did not fix the manifold edges,=BLOB=,celiib@10.28.0.81,f031c4c9faf9,33618,732395,2019-07-14 03:50:38
__neuron_skeleton_stitched,188fa44ea025fb0d461d0330a1804295,error,=BLOB=,Exception: neuron648518346349487734 did not fix the manifold edges,=BLOB=,celiib@10.28.0.81,f031c4c9faf9,33645,732355,2019-07-14 03:50:37
__neuron_skeleton_stitched,2ca10f434d9cb4fcd6c10ead409c5b62,error,=BLOB=,ValueError: string is not a file: /notebooks/Pass_2_Sk_Neurons/copies/pymesh_NEURONS/neuron_648518346349500431_piece_0_mls.off,=BLOB=,celiib@10.28.0.81,f031c4c9faf9,33641,732336,2019-07-14 03:51:26
__neuron_skeleton_stitched,31503451191d0a9840dfb058ff62023f,error,=BLOB=,IndexError: index 0 is out of bounds for axis 0 with size 0,=BLOB=,celiib@10.28.0.81,f031c4c9faf9,33594,732278,2019-07-14 03:27:26
__neuron_skeleton_stitched,349d0dfea202b5556f7047e8a2b97bbd,error,=BLOB=,ValueError: string is not a file: /notebooks/Pass_2_Sk_Neurons/copies/pymesh_NEURONS/neuron_648518346349482676_piece_2_mls.off,=BLOB=,celiib@10.28.0.81,f031c4c9faf9,33576,732359,2019-07-14 03:51:12
__neuron_skeleton_stitched,3925b2182d53283101a42e22064bc855,error,=BLOB=,Exception: neuron648518346349505696 did not fix the manifold edges,=BLOB=,celiib@10.28.0.81,f031c4c9faf9,33714,732280,2019-07-14 03:50:58
__neuron_skeleton_stitched,3c71c781d1601b93781e5666b10328cf,error,=BLOB=,ValueError: string is not a file: /notebooks/Pass_2_Sk_Neurons/copies/pymesh_NEURONS/neuron_648518346349500939_piece_1_mls.off,=BLOB=,celiib@10.28.0.81,f031c4c9faf9,33575,732259,2019-07-14 03:50:38
__neuron_skeleton_stitched,3e3ea82f7d993717d95f21c4afc984b6,error,=BLOB=,Exception: neuron648518346349499783 did not fix the manifold edges,=BLOB=,celiib@10.28.0.81,f031c4c9faf9,33574,732287,2019-07-14 03:50:40
__neuron_skeleton_stitched,42439e3d2dcc5628b7de92eb2589dc90,error,=BLOB=,IndexError: index 0 is out of bounds for axis 0 with size 0,=BLOB=,celiib@10.28.0.81,f031c4c9faf9,33605,732270,2019-07-14 03:27:26


In [ ]:
schema.jobs

In [ ]:
#ta3p100.FilteredSkeleton.describe()
#ta3p100.FilteredSkeletonMinusSoma.describe()
#ta3p100.FilteredSkeletonMinusSoma.drop()


In [ ]:
#function that takes in a 3x3 array of coordinates for faces and returns triangles and vertices
def index_unique_rows(full_coordinate_array):
    """
    Separates an array of nested coordinate rows into an array of unique rows and and index array.
    """
    vertices, flat_idx = np.unique(full_coordinate_array.reshape(-1, full_coordinate_array.shape[-1]), axis=0, return_inverse=True)
    return vertices, flat_idx.reshape(-1, full_coordinate_array.shape[-2])





In [ ]:
#will take in and populate the soma table based on the key it gets
def soma_verts_faces(query_key):
    
    table=""
    vertices_soma,triangles_soma = (pinky.CompartmentFinal.ComponentFinal() & query_key
                                    & "compartment_type='Soma'").fetch("vertex_indices","triangle_indices")

    if len(vertices_soma) > 0:
        print("Soma found in Exhitatory")
        #get the regular mesh from CleansedMesh
        vertices_mesh,triangles_mesh = (pinky.PymeshfixDecimatedExcitatoryStitchedMesh & query_key).fetch("vertices","triangles")
    else:
        vertices_soma,triangles_soma = (pinky.CompartmentOrphan.ComponentOrphan() & query_key & "compartment_type='Soma'").fetch("vertex_indices","triangle_indices")
        if len(vertices_soma) > 0:
            print("Soma found in Orphans")
            vertices_mesh,triangles_mesh = (pinky.Decimation35OrphanStitched & query_key).fetch("vertices","triangles")
        else:
            print("No Soma exists for " + str(query_key["segment_id"]))
            return np.array([]),np.array([])
            
    ts_flatten = np.hstack(triangles_soma).astype("int64")

    vertices_real = vertices_mesh[0]
    triangles_real = triangles_mesh[0]

    ts_stack_whole = vertices_real[triangles_real[ts_flatten]]

    vertices_whole, triangles_whole = index_unique_rows(ts_stack_whole)
    return vertices_whole, triangles_whole

In [ ]:
#will take in and populate the soma table based on the key it gets
def soma_verts_faces_leftover(query_key):
    
    table=""
    vertices_soma,triangles_soma = (pinky.LeftoverCompartmentFinal.LeftoverComponentFinal() & query_key
                                    & "compartment_type='Soma'").fetch("vertex_indices","triangle_indices")

    if len(vertices_soma) > 0:
        print("Soma found in Exhitatory Leftover")
        #get the regular mesh from CleansedMesh
        vertices_mesh,triangles_mesh = (pinky.Exc & query_key).fetch("vertices","triangles")
    else:
        print("No Soma exists for leftover mesh in seg: " + str(query_key["segment_id"]))
        return np.array([]),np.array([])
            
    ts_flatten = np.hstack(triangles_soma).astype("int64")

    vertices_real = vertices_mesh[0]
    triangles_real = triangles_mesh[0]

    ts_stack_whole = vertices_real[triangles_real[ts_flatten]]

    vertices_whole, triangles_whole = index_unique_rows(ts_stack_whole)
    return vertices_whole, triangles_whole

In [ ]:
def filter_edges_by_bounding_box(edges,max_bb_zone,min_bb_zone):
    #can just use bounding box function to get rid of any inside edges
    filtered_remaining = list()

    for i,e in enumerate(edges):
        #print(e)
        if min(e[0][0],e[1][0])>max_bb_zone[0]:
            #print("minx>maxx")
            filtered_remaining.append(e)
            
            continue

        if max(e[0][0],e[1][0])<min_bb_zone[0]:
            #print("maxx<minx")
            filtered_remaining.append(e)
            continue

        if min(e[0][1],e[1][1])>max_bb_zone[1]:
            #print("miny>maxy")
            filtered_remaining.append(e)
            continue

        if max(e[0][1],e[1][1])<min_bb_zone[1]:
            #print("maxy<miny")
            filtered_remaining.append(e)
            continue

        if min(e[0][2],e[1][2])>max_bb_zone[2]:
            #print("minz>maxz")
            filtered_remaining.append(e)
            continue

        if max(e[0][2],e[1][2])<min_bb_zone[2]:
            #print("maxz<minz")
            filtered_remaining.append(e)
            continue

        #filtered_edges.append(e)

    return np.array(filtered_remaining)

In [ ]:
exhitatory_with_somas = dj.U("segment_id","segmentation") & ((pinky.CompartmentFinal.ComponentFinal() & "compartment_type='Soma'").proj("segment_id"))
#exhitatory_with_somas

In [ ]:
orphans_with_somas = dj.U("segment_id","segmentation") & ((pinky.CompartmentOrphan.ComponentOrphan() & "compartment_type='Soma'").proj("segment_id"))
#orphans_with_somas


In [ ]:
total_soma = ((dj.U("segment_id","segmentation") & ((pinky.CompartmentOrphan.ComponentOrphan() & "compartment_type='Soma'").proj("segment_id")).proj()) + 
(dj.U("segment_id","segmentation") & ((pinky.CompartmentFinal.ComponentFinal() & "compartment_type='Soma'").proj("segment_id"))
.proj()))
    
#total_soma



In [ ]:
#(ta3p100.FilteredSkeleton & total_soma)

In [ ]:
total_neurons_with_somas = exhitatory_with_somas.proj() + orphans_with_somas.proj()
print((len(total_neurons_with_somas),35 +318))


In [ ]:
def get_soma_mesh(key):
    """
    Will return a trimesh object representing the mesh for that soma
    """
    
    #get the mesh
    verts,faces = (pinky.Mesh & key).fetch1("vertices","triangles")
    current_mesh = trimesh.Trimesh(verts,faces,process=False)
    
    #get the labels for the mesh
    face_labels = (pinky.UndecimatedNeuronLabels & key).fetch1("coarse_triangles_labels")
    
    #get the indexes of the soma
    soma_indexes = np.where(face_labels == 5)
    
    if not soma_indexes.any():
        return trimesh.Trimesh(np.array([]),np.array([]))
    
    #get a submesh
    soma_mesh = current_mesh.submesh([soma_indexes],append=True)
    return soma_mesh

In [ ]:
@schema
class FilteredSkeletonMinusSoma(dj.Computed):
    definition="""
    -> pinky.FilteredNeuronSkeleton
    ---
    vertices              :longblob #vertex coordinates of soma mesh
    triangles             :longblob #faces for soma mesh
    edges                  :longblob #edges of skeelton after soma edges removed
    n_edges               :int unsigned #number of edges of skeelton after soma edges removed
    soma_bounding_corners :blob #bounding box corners for the soma mesh
    """
    
    key_source = pinky.FilteredNeuronSkeleton & ((dj.U("segment_id","segmentation") & ((pinky.CompartmentOrphan.ComponentOrphan() & "compartment_type='Soma'").proj("segment_id")).proj()) + 
                (dj.U("segment_id","segmentation") & ((pinky.CompartmentFinal.ComponentFinal() & "compartment_type='Soma'").proj("segment_id")).proj()))
    #how you get the date and time  datetime.datetime.now()
    
    def make(self, key):
        print()
        print()
        print(str(key["segment_id"])+ ":")
        global_start_time = time.time()
        #create return key
        return_key = key.copy()

        #pull down the skeleton for the mesh
        skeleton_data = (pinky.FilteredNeuronSkeleton() & key).fetch(as_dict=True)[0]
        
        #get the vertices and triangles for the Soma
        start_time = time.time()
        
        mesh = get_soma_mesh(key)
        
        if len(mesh.faces) < 2:
            
            print("No Soma Found")

            #return_key["soma_exist"] = False
            return_key["vertices"] = mesh.vertices
            return_key["triangles"] = mesh.faces
            return_key["edges"] = skeleton_data["edges"]
            return_key["n_edges"] = skeleton_data["n_edges"]
            return_key["soma_bounding_corners"] = np.array([])

            self.insert(return_key,skip_duplicates=True)
        
        """
        vertices_whole, triangles_whole = soma_verts_faces(key)
        
        #look for any soma meshes in the leftover meshes
        vertices_whole_leftover, triangles_whole_leftover = soma_verts_faces_leftover(key)
        
        print(f"Step 1: extracted Soma Mesh = {time.time()-start_time}")

        #if no soma portion was found then just write regular skeleton
        if not vertices_whole.any() and not vertices_whole_leftover.any():
            
            print("No Soma Found")

            #return_key["soma_exist"] = False
            return_key["vertices"] = vertices_whole
            return_key["triangles"] = triangles_whole
            return_key["edges"] = skeleton_data["edges"]
            return_key["n_edges"] = skeleton_data["n_edges"]
            return_key["soma_bounding_corners"] = np.array([])

            self.insert(return_key,skip_duplicates=True)

        #just need to strip the portions of the skeleton that are inside of the mesh

        #find the bounding box
        start_time = time.time()
        mesh_regular = trimesh_io.Mesh(vertices=vertices_whole, faces=triangles_whole)
        mesh_leftover = trimesh_io.Mesh(vertices=vertices_whole_leftover, faces=triangles_whole_leftover)
        mesh = mesh_regular + mesh_leftover
        
        """
        
        min_bb = np.array(mesh.bounding_box.vertices).min(0)
        max_bb = np.array(mesh.bounding_box.vertices).max(0)
        print(f"Step 2: Calculated Bounding Box = {time.time()-start_time}")

        start_time = time.time()
        #get the filtered edges according to bounding box:
        filtered_edges_postsyn = filter_edges_by_bounding_box(skeleton_data["edges"],max_bb,min_bb)

        print(f"Step 3: filtering edges = {time.time()-start_time}")

        

        #write off the new data to the table
        #return_key["soma_exist"] = True
        
        start_time = time.time()
        
        return_key["vertices"] = mesh.vertices
        return_key["triangles"] = mesh.faces
        return_key["edges"] = filtered_edges_postsyn
        return_key["n_edges"] = filtered_edges_postsyn.shape[0]
        return_key["soma_bounding_corners"] = np.array((min_bb,max_bb))
        
#         print(return_key)
#         print(return_key.keys())
#         print("len(return_key.keys()) = " + str(len(return_key.keys())))
        
#         for k in return_key.keys():
#             print("type(return_key["+k+"])=" + str(type(return_key[k])))
        
        self.insert1(return_key,skip_duplicates=True,ignore_extra_fields=True)
        print(f"Step 4: Inserted Key = {time.time()-start_time}")
        print(f"Total time = {time.time()-global_start_time}")
    
    

In [ ]:
notebook_start_time = time.time()
FilteredSkeletonMinusSoma.populate(reserve_jobs=True)
print(f"Entire Notebook = {time.time()-notebook_start_time}")

# debugging

In [ ]:
#(schema.jobs & "table_name='__filtered_skeleton_minus_soma'").delete()

In [ ]:
#FilteredSkeletonMinusSoma.drop()

In [ ]:
# notebook_start_time = time.time()
# FilteredSkeletonMinusSoma.populate(reserve_jobs=True)
# print(f"Entire Notebook = {time.time()-notebook_start_time}")

In [ ]:
# ta3p100.CleansedMeshOrphan & (FilteredSkeletonMinusSoma() & "n_edges=0").proj()

In [ ]:
# #investigating the error: arrays used as indices must be of integer (or boolean) type
# query_key = dict(segmentation=2,segment_id=648518346341378749)

# table=""
# vertices_soma,triangles_soma = (ta3p100.CompartmentFinal.ComponentFinal() & query_key
#                                 & "compartment_type='Soma'").fetch("vertex_indices","triangle_indices")

# if len(vertices_soma) > 0:
#     print("Soma found in Exhitatory")
#     #get the regular mesh from CleansedMesh
#     vertices_mesh,triangles_mesh = (ta3p100.CleansedMesh & query_key).fetch("vertices","triangles")
# else:
#     vertices_soma,triangles_soma = (ta3p100.CompartmentOrphan.ComponentOrphan() & query_key).fetch("vertex_indices","triangle_indices")
#     if len(vertices_soma) > 0:
#         print("Soma found in Orphans")
#         vertices_mesh,triangles_mesh = (ta3p100.CleansedMeshOrphan & query_key).fetch("vertices","triangles")
#     else:
#         print("No Soma exists for " + str(query_key["segment_id"]))
#         #return np.array([]),np.array([])

# ts_flatten = np.hstack(triangles_soma).astype("int64")
# type(ts_flatten[0])

# vertices_real = vertices_mesh[0]
# triangles_real = triangles_mesh[0]

# ts_stack_whole = vertices_real[triangles_real[ts_flatten]]

# vertices_whole, triangles_whole = index_unique_rows(ts_stack_whole)
# #return vertices_whole, triangles_whole